# Initial build up of Neural Network Model

In [8]:
import json
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from keras import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Get the data as Numpy arrays
train_set = pd.read_csv('../data/subtrain/train_1.tsv', sep='\t')
train_set['cross_city'] = train_set['cross_city'].astype('int')
train_set['cross_state'] = train_set['cross_state'].astype('int')
valid_set = pd.read_csv('../data/subtrain/valid_1.tsv', sep='\t')
valid_set['cross_city'] = valid_set['cross_city'].astype('int')
valid_set['cross_state'] = valid_set['cross_state'].astype('int')

In [4]:
train_set[:5]
train_set.shape
valid_set.shape

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt,package_size,weight,tz_dis,dis,cross_city,cross_state,acc_hour,pay_hour,acc_date,target
0,1503096,2,0.0,2,9,10,21.00,1,1,0,0.0,0.0,1.778966,1,1,7,19,20190104,3
1,1503097,0,3.5,3,5,3,1.40,1,0,0,0.0,-1.0,0.823542,1,1,21,9,20190107,2
2,1503098,7,0.0,2,5,4,0.99,1,0,0,13.0,-1.0,1.184160,1,1,15,8,20190109,2
3,1503099,5,6.9,2,5,0,9.00,1,1,0,0.0,-4.0,3.897320,1,1,14,15,20190128,2
4,1503100,0,3.5,3,5,6,28.60,1,0,0,3.0,-1.0,1.003335,1,1,14,13,20190102,3


(5889444, 19)

(654382, 19)

In [5]:
x_train_valid = pd.concat([train_set, valid_set])

In [6]:
c1 = pd.get_dummies(x_train_valid.shipment_method_id, prefix='sm')
c2 = pd.get_dummies(x_train_valid.category_id, prefix='ci')
c3 = pd.get_dummies(x_train_valid.package_size, prefix='ps')
c4 = pd.get_dummies(x_train_valid.cross_city, prefix='cc')
c5 = pd.get_dummies(x_train_valid.cross_state, prefix='cs')
x_train_valid = pd.concat([x_train_valid.drop(['record_number', 'target', 'shipment_method_id', 'category_id', 'package_size', 'cross_city', 'cross_state'], axis=1), 
               c1, c2, c3, c4, c5], axis=1)

In [7]:
x_train = x_train_valid[:train_set.shape[0]]
x_valid = x_train_valid[train_set.shape[0]:]
x_train.shape
x_valid.shape

(5889444, 82)

(654382, 82)

In [8]:
y_train = train_set.target
y_valid = valid_set.target

print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

(5889444, 82) (5889444,) (654382, 82) (654382,)


In [9]:
# Build a simple model
inputs = keras.Input(shape=(82,))
x = layers.Dense(512, activation="sigmoid")(inputs)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="sigmoid")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="tanh")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="tanh")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 82)]              0         
                                                                 
 dense (Dense)               (None, 512)               42496     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0     

In [10]:
model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 82),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'units': 512,
    'activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'dense',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Dropout',
   'config': {'name': 'dropout',
    'trainable': True,
    'dtype': 'float32',
    'rate': 0.4,
    'noise_shape': None,
    'seed': None},
   'name': 'dropout',
   'inbound_nod

In [5]:
def custom_asymmetric_eval(y_train, preds):
    nd_preds = tf.math.round(preds)
    residual = y_train - nd_preds
    loss = tf.where(residual < 0, residual * -0.6, residual * 0.4) 
    return tf.math.reduce_mean(loss)

In [1]:
from keras.callbacks import CSVLogger

2021-12-30 02:38:35.652431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-30 02:38:35.652492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='custom_asymmetric_eval', patience=3)
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error", metrics = [custom_asymmetric_eval])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='custom_asymmetric_eval', patience=3),
                 CSVLogger('../result/Keraslog.csv', append=False, separator=';')]
batch_size = 256
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1, validation_data=(x_valid, y_valid),callbacks=[callbacks])

In [45]:
logstr = pd.read_csv("../result/Keraslog.csv", sep = ';')


0.4852772355079651

In [46]:
loss_and_output = []
loss_and_output.append(float(logstr.val_custom_asymmetric_eval))

In [47]:
loss_and_output
lao = np.array([1 / x for x in loss_and_output])

lao = lao / lao.sum()
lao[0]

[0.4852772355079651]

1.0

In [20]:
# load model and predict
from keras.models import load_model
#custom_objects = {"custom_metric": custom_asymmetric_eval}
model = load_model('../para/nn_9.h5', custom_objects={'custom_asymmetric_eval': custom_asymmetric_eval})

In [31]:
quiz_set = pd.read_csv('../data/parsed_quiz_cat.tsv', sep='\t')

In [32]:
quiz_set.drop(['record_number'], axis=1, inplace=True)

In [33]:
quiz_set.shape

(2500000, 82)

In [34]:
y_pred = model.predict(quiz_set)

2021-12-30 21:33:09.192525: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1640000000 exceeds 10% of free system memory.


In [38]:
y_pred

array([[2.9306798],
       [2.9306798],
       [2.9306798],
       ...,
       [2.9306798],
       [2.9306798],
       [2.9306798]], dtype=float32)